In [1]:
import nltk


In [2]:
import re

In [3]:
import pandas as pd 

In [4]:
import numpy as np 

In [5]:
from nltk.corpus import stopwords

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
from nltk.stem import PorterStemmer ,WordNetLemmatizer

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report

# data gathering

In [13]:
df=pd.read_csv("News_dataset.csv")

In [14]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


# data anlysis

In [16]:
df['label'].value_counts()

label
1    10413
0    10387
Name: count, dtype: int64

In [17]:
df.shape

(20800, 5)

In [18]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
df=df.dropna()

In [20]:
df.isna().sum()


id        0
title     0
author    0
text      0
label     0
dtype: int64

In [21]:
df.shape

(18285, 5)

In [22]:
df.reset_index(inplace=True)

In [23]:
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [25]:
df=df.drop(['id','text','author'],axis=1)

In [26]:
df.head()

,index,title,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,2,Why the Truth Might Get You Fired,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,1
4,4,Iranian woman jailed for fictional unpublished...,1


# data Preprocessing

## 1 tokenization

In [27]:
sample_data="the Quick brown fox Jumps over The lazy dog"
sample_data=sample_data.split()
sample_data

['the', 'Quick', 'brown', 'fox', 'Jumps', 'over', 'The', 'lazy', 'dog']

## 2. make lowercasse

In [28]:
sample_data=[data.lower() for data in sample_data]
sample_data

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

## 3. remove stopwards

In [29]:
stopwords_list = stopwords.words('english')
sample_data=[data for data in sample_data if data not in stopwords_list]
print(sample_data)
print(len(sample_data))

['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']
6


## 4. stemming

In [30]:
ps= PorterStemmer()
sample_data_stemming= [ps.stem(data) for data in sample_data]
print(sample_data_stemming)

['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']


## 5. lemmitization

In [31]:
lm= WordNetLemmatizer()

In [32]:
sample_data_lemma=[lm.lemmatize(data) for data in sample_data]

In [33]:
print(sample_data_lemma)

['quick', 'brown', 'fox', 'jump', 'lazy', 'dog']


In [34]:
lm= WordNetLemmatizer()
corpus = []
for i in range (len(df)):
    review=re.sub('^a-zA-Z0-9',' ',df['title'][i])
    review=review.lower()
    review= review.split()
    review= [lm.lemmatize(x) for x in review if x not in (stopwords.words('english'))]
    review= " ".join(review)
    corpus.append(review)

In [35]:
len(corpus)

18285

In [36]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [37]:

corpus[0]


'house dem aide: didn’t even see comey’s letter jason chaffetz tweeted'

## converting our text into vector (vectorization)

In [38]:
corpus_strings = [str(doc) for doc in corpus]
tf=TfidfVectorizer()

In [39]:
x=tf.fit_transform(corpus)
x_array=x.toarray()


In [40]:
x

<18285x19259 sparse matrix of type '<class 'numpy.float64'>'
	with 161420 stored elements in Compressed Sparse Row format>

In [41]:
y=df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

## splittng data into train andtest split

In [42]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = 0.3,random_state=10,stratify=y)

In [43]:
num_samples_train, num_features_train = x_train.shape
num_samples_y_train = len(y_train)

print("Number of samples in X_train:", num_samples_train)
print("Number of features in X_train:", num_features_train)
print("Number of samples in y_train:", num_samples_y_train)

Number of samples in X_train: 12799
Number of features in X_train: 19259
Number of samples in y_train: 12799


In [44]:
x_train.shape, y_train.shape

((12799, 19259), (12799,))

In [45]:
x_test.shape,y_test.shape

((5486, 19259), (5486,))

## model building 

In [46]:
rf= RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

## model evaluation

In [72]:
y_pred = rf.predict(x_test)
accuracy_score_ = accuracy_score(y_test,y_pred) 
accuracy_score_

0.9391177542836311

In [63]:
class Evaluation:
    def __init__(self, model, x_train, x_test, y_train, y_test):
        self.model = model
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test

    def train_evaluation(self):
        y_pred_train = self.model.predict(self.x_train)

        acc_scr_train = accuracy_score(self.y_train, y_pred_train)
        print("Accuracy score on Training dataset:", acc_scr_train)

        con_mat_train = confusion_matrix(self.y_train, y_pred_train)
        print("Confusion matrix on Training dataset:\n", con_mat_train)

        class_rep_train = classification_report(self.y_train, y_pred_train)
        print("Classification report on Training dataset:\n", class_rep_train)

    def test_evaluation(self):
        y_pred_test = self.model.predict(self.x_test)

        acc_scr_test = accuracy_score(self.y_test, y_pred_test)  # Use a different variable name
        print("Accuracy score on Testing dataset:", acc_scr_test)

        con_mat_test = confusion_matrix(self.y_test, y_pred_test)
        print("Confusion matrix on Testing dataset:\n", con_mat_test)

        class_rep_test = classification_report(self.y_test, y_pred_test)
        print("Classification report on Testing dataset:\n", class_rep_test)








## checking the accuracy on training dataset 

In [73]:
Evaluation(rf,x_train,x_test,y_train,y_test).train_evaluation()

Accuracy score on Training dataset: 1.0
Confusion matrix on Training dataset:
 [[7252    0]
 [   0 5547]]
Classification report on Training dataset:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7252
           1       1.00      1.00      1.00      5547

    accuracy                           1.00     12799
   macro avg       1.00      1.00      1.00     12799
weighted avg       1.00      1.00      1.00     12799



## checking the accuracy on testing dataset 

In [74]:
Evaluation(rf,x_train,x_test,y_train,y_test).test_evaluation()

Accuracy score on Testing dataset: 0.9391177542836311
Confusion matrix on Testing dataset:
 [[2834  275]
 [  59 2318]]
Classification report on Testing dataset:
               precision    recall  f1-score   support

           0       0.98      0.91      0.94      3109
           1       0.89      0.98      0.93      2377

    accuracy                           0.94      5486
   macro avg       0.94      0.94      0.94      5486
weighted avg       0.94      0.94      0.94      5486



## creating a pprediction pipeline

In [95]:
class preprocessing:

    def __init__(self,data):
        self.data=data
    def test_preprocessing_user(self):
        lm= WordNetLemmatizer()
        pred_data= [self.data]
        preprocess_data = []
        for data in pred_data:
            review=re.sub('^a-zA-Z0-9',' ',data)
            review=review.lower()
            review= review.split()
            review= [lm.lemmatize(x) for x in review if x not in (stopwords.words('english'))]
            review= " ".join(review)
            preprocess_data.append(review)
        return preprocess_data

In [96]:
df['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [98]:
data ='FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
preprocessing(data).test_preprocessing_user()

['flynn: hillary clinton, big woman campus - breitbart']

In [101]:
class Prediction:
    
    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model
        
    def prediction_model(self):
        preprocess_data = preprocessing(self.pred_data).test_preprocessing_user()
        data = tf.transform(preprocess_data)
        prediction = self.model.predict(data)
        
        if prediction [0] == 0 :
            return "The News Is Fake"
        
        else:
            return "The News Is Real"
        

In [ ]:
class prediction:

     def __init__(self,pred_data):
         self.pred_data=pred_data
     def prediction_model(self):
         preprocess_data =  preprocessing(self.pred_data).text_preprocessing_user()

In [102]:
data = 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
Prediction(data,rf).prediction_model()

'The News Is Fake'

In [103]:
df['title'][3]

'15 Civilians Killed In Single US Airstrike Have Been Identified'

In [104]:
user_data = '15 Civilians Killed In Single US Airstrike Have Been Identified' 
Prediction(user_data,rf).prediction_model()

'The News Is Real'

In [105]:
df['title'][4]

'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [106]:
user_data = 'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'
Prediction(user_data,rf).prediction_model()

'The News Is Real'

In [111]:
df['title'][19]

'Rob Reiner: Trump Is ’Mentally Unstable’ - Breitbart'

In [112]:
user_data = 'Rob Reiner: Trump Is ’Mentally Unstable’ - Breitbart'
Prediction(user_data,rf).prediction_model()

'The News Is Fake'

In [113]:
df['title'][21]

'Nukes and the UN: a Historic Treaty to Ban Nuclear Weapons'

In [114]:
user_data = 'Nukes and the UN: a Historic Treaty to Ban Nuclear Weapons'
Prediction(user_data,rf).prediction_model()

'The News Is Real'